In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path, PosixPath
from tqdm import tqdm
import sys
import glob
import datetime
from typing import List
import json

In [2]:
CREATE_CSV_FILES = True
CAMELS_COMBINED = True
root_path = "/content/gdrive/MyDrive/Colab Datasets/Hydrology/gb"

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Create csv Files



## Read Timeseries Files

In [3]:
# read dynamic
def to_datetime(s):
  date = s.split('-')
  year = int(date[0])
  month = int(date[1])
  day = int(date[2])
  d = datetime.date(year, month, day)
  return d


def read_timeseries_data(data_dir: str):
  files = list(Path(data_dir).glob('CAMELS_GB_hydromet_timeseries_*.csv'))
  input = []

  for csv in files:
    file_name = os.path.basename(csv)
    names = file_name.split('_')

    df_input = pd.read_csv(csv, dtype=None)
    df_input['basin_id'] = int(names[4])
    df_input['date'].apply(to_datetime)
    input.append(df_input)

  timeseries_data = pd.concat(input, axis=0, copy=False)

  return timeseries_data

In [ ]:
if CREATE_CSV_FILES:
  timeseries_dir = root_path + "/data/timeseries"
  timeseries_data = read_timeseries_data(timeseries_dir)
  forcing_data = timeseries_data[['date', 'basin_id', 'precipitation', 'temperature']]
  discharge_data = timeseries_data[['date', 'basin_id', 'discharge_spec']]

  forcing_data.to_csv(root_path + "/forcing_data_gb.csv", index=False)
  discharge_data.to_csv(root_path + "/discharge_data_gb.csv", index=False)

## Read Attributes Files

In [5]:
def read_attributes(data_dir: str):
    files = list(Path(data_dir).glob('CAMELS_GB_*_attributes.csv'))

    data_list = []

    pbar = tqdm(files, file=sys.stdout, position=0)
    for fpath in pbar:
        pbar.set_description("process " + fpath.name)
        data = pd.read_csv(fpath)
        data['gauge_id'].astype('int64')

        data_list.append(data)

    attributes = pd.concat(data_list, axis=1, copy=False)

    return attributes

In [6]:
if CREATE_CSV_FILES:
  attributes_dir = root_path + "/data"
  attr = read_attributes(attributes_dir)
  attr.to_csv(root_path + "/attributes_gb.csv", index=False)
  attr = attr.astype({"gauge_id": int}, errors='raise')
  attr

process CAMELS_GB_topographic_attributes.csv: 100%|██████████| 8/8 [00:04<00:00,  1.87it/s]


# Read csv Files

In [7]:
BASIN_COL = 'basin_id'
DATE_COL = 'date'

In [8]:
# read csvs
attr_p = pd.read_csv(root_path + "/attributes_gb.csv")
discharge_p = pd.read_csv(root_path + "/discharge_data_gb.csv")
forcing_p = pd.read_csv(root_path + "/forcing_data_gb.csv")

In [9]:
forcing_p.dtypes

date              object
basin_id           int64
precipitation    float64
temperature      float64
dtype: object

In [10]:
locs = np.union1d(np.union1d(forcing_p[BASIN_COL], discharge_p[BASIN_COL]), attr_p['gauge_id'])
Nloc = len(locs)

print('Initial Nloc', Nloc)

Initial Nloc 671


## Sanity Check

In [11]:
def remove_basins_globally(basins: List[int]):
  global discharge_p
  global forcing_p
  global attr_p
  global locs
  global Nloc

  print("removing", basins)

  discharge_p.drop(discharge_p[discharge_p[BASIN_COL].isin(basins)].index, inplace=True, errors='ignore')
  forcing_p.drop(forcing_p[forcing_p[BASIN_COL].isin(basins)].index, inplace=True, errors='ignore')
  attr_p.drop(attr_p[attr_p['gauge_id'].isin(basins)].index, inplace=True, errors='ignore')

  locs = np.setdiff1d(locs, basins)
  Nloc = len(locs)
  print('new Nloc', Nloc)

In [12]:
intersect = np.intersect1d(np.intersect1d(forcing_p[BASIN_COL], discharge_p[BASIN_COL]), attr_p['gauge_id'])

diff = np.setdiff1d(locs, intersect)
print("diff", diff)

if len(diff) > 0:
  remove_basins_globally(diff)

diff []


In [13]:
def find_time_range(df: pd.DataFrame):
  groups = df[[BASIN_COL, DATE_COL]].groupby([BASIN_COL])
  counts = groups.count()

  abnormal_cols = counts[counts[DATE_COL] < 1000].index.values

  if len(abnormal_cols) > 0:
    print('abnormal cols', abnormal_cols)
    remove_basins_globally(abnormal_cols)
    groups = df[[BASIN_COL, DATE_COL]].groupby([BASIN_COL])

  mins = groups.min()
  maxs = groups.max()

  min_d  = np.max(mins.values)
  max_d  = np.min(maxs.values)
  print("range", min_d, max_d)

  return min_d, max_d

In [14]:
min1, max1 = find_time_range(discharge_p)
min2, max2 = find_time_range(forcing_p)

range 1970-10-01 2015-09-30
range 1970-10-01 2015-09-30


In [15]:
InitialDate = max(min1, min2)
InitialDate = "1989-10-02"
print("InitialDate ", InitialDate)


EndDate = min(max1, max2)
EndDate = "2008-12-31"
print("EndDate ", EndDate)

InitialDate  1989-10-02
EndDate  2008-12-31


In [16]:
# filter data sets based on the InitialDate and EndDate
discharge_p.drop(discharge_p[(discharge_p[DATE_COL] < InitialDate) | (discharge_p[DATE_COL] > EndDate)].index,
                 inplace=True, errors='ignore')
forcing_p.drop(forcing_p[(forcing_p[DATE_COL] < InitialDate) | (forcing_p[DATE_COL] > EndDate)].index,
                 inplace=True, errors='ignore')

## Dynamic Data

In [17]:
input_p = forcing_p.merge(discharge_p, on=[BASIN_COL, DATE_COL], copy=True, validate="1:1")
input_p

,date,basin_id,precipitation,temperature,discharge_spec
0,1989-10-02,10002,0.00,8.71,0.27
1,1989-10-03,10002,0.00,9.63,0.27
2,1989-10-04,10002,0.39,11.12,0.27
3,1989-10-05,10002,5.69,11.32,0.30
4,1989-10-06,10002,2.52,9.24,0.31
...,...,...,...,...,...
4717796,2008-12-27,96004,0.04,-1.20,1.08
4717797,2008-12-28,96004,0.00,-2.45,0.91
4717798,2008-12-29,96004,0.00,-2.75,0.82
4717799,2008-12-30,96004,0.02,-6.09,0.56


In [18]:
input_p.rename(columns = {'date':'Year_Mnth_Day', 'basin_id':'basin_id', 'precipitation':'prcp(mm/day)', 'temperature':'tmean(C)', 'discharge_spec':'QObs(mm/d)'}, inplace = True)

In [19]:
input_p.sort_values(["Year_Mnth_Day", BASIN_COL], inplace=True, ignore_index=True)

In [20]:
input_p.sort_values(["Year_Mnth_Day", BASIN_COL], inplace=True, ignore_index=True)
input_p['QObs(mm/d)'] = input_p['QObs(mm/d)'].fillna(input_p['QObs(mm/d)'].mean())
input_p['prcp(mm/day)'] = input_p['prcp(mm/day)'].astype('float')
input_p = input_p[~(input_p['Year_Mnth_Day'] < '1989-10-02')]
input_p = input_p[~(input_p['Year_Mnth_Day'] > '2008-12-31')]
input_p['Year_Mnth_Day'] = pd.to_datetime(input_p['Year_Mnth_Day'], format='%Y-%m-%d')

In [21]:
assert len(input_p) == len (forcing_p) and len(input_p) == len (discharge_p)

In [22]:
BasicInputTimeSeries = input_p.to_numpy()

In [23]:
print(BasicInputTimeSeries.shape)
print(BasicInputTimeSeries[0])

(4717801, 5)
[Timestamp('1989-10-02 00:00:00') 1001 0.02 8.5 1.9407891071608745]


## Static Data

In [24]:
INVALID_ATTR= []

if CAMELS_COMBINED:
  INVALID_ATTR = [
      'gauge_name', 'gauge_id', 'gauge_easting', 'gauge_northing', 'gauge_elevation', 'dpsbar', 'elev_min', 'elev_10',
      'elev_50', 'elev_90', 'elev_max', 'baseflow_index_ceh', 'dwood_perc', 'ewood_perc', 'grass_perc', 'shrub_perc', 'crop_perc',
      'urban_perc', 'inwater_perc', 'bares_perc', 'gauge_elev'
  ]

# else:
#   # Test fitting GB data into US model
#   INVALID_ATTR = [
#       'gauge_name', 'gauge_id', 'gauge_elev', 'area', 'gauge_easting', 'gauge_northing', 'gauge_elevation', 'dpsbar', 'elev_min', 'elev_10',
#       'elev_50', 'elev_90', 'elev_max', 'baseflow_index_ceh'
#   ]

In [25]:
gauge_id = attr_p['gauge_id'].tolist()

In [26]:
filtered_attr = attr_p.copy()
filtered_attr.drop(INVALID_ATTR, axis=1, inplace=True, errors='ignore')
filtered_attr.insert(0, 'gauge_id', gauge_id)
filtered_attr.dtypes

gauge_id             int64
inter_high_perc    float64
inter_mod_perc     float64
inter_low_perc     float64
frac_high_perc     float64
                    ...   
gauge_id.7           int64
gauge_lat          float64
gauge_lon          float64
area               float64
elev_mean          float64
Length: 134, dtype: object

In [27]:
# Re-order Columns
filtered_attr = filtered_attr[['gauge_id', 'p_mean', 'pet_mean', 'p_seasonality', 'frac_snow', 'aridity', 'high_prec_freq',
                               'high_prec_dur', 'high_prec_timing', 'low_prec_freq', 'low_prec_dur', 'low_prec_timing', 'q_mean',
                               'runoff_ratio', 'slope_fdc', 'baseflow_index', 'stream_elas', 'Q5', 'Q95', 'high_q_freq',
                               'high_q_dur', 'low_q_freq', 'low_q_dur', 'zero_q_freq', 'hfd_mean', 'gauge_lat', 'gauge_lon',
                               'elev_mean', 'area', 'dom_land_cover']]

In [28]:
assert len(filtered_attr) == Nloc

In [29]:
# Categorical variable encoding
cols = filtered_attr.columns
categorical_cols = filtered_attr.select_dtypes(exclude = ['number']).columns

for cat_col in categorical_cols:
  num_cat = filtered_attr[cat_col].value_counts().count()
  filtered_attr[cat_col] = (filtered_attr[cat_col].astype('category').cat.codes)/num_cat

  print('Processed ', cat_col)

Processed  high_prec_timing
Processed  low_prec_timing
Processed  dom_land_cover


<ipython-input-29-9d2cd170c9d6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_attr[cat_col] = (filtered_attr[cat_col].astype('category').cat.codes)/num_cat
<ipython-input-29-9d2cd170c9d6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_attr[cat_col] = (filtered_attr[cat_col].astype('category').cat.codes)/num_cat
<ipython-input-29-9d2cd170c9d6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [30]:
filtered_attr.isna().sum()

gauge_id            0
p_mean              0
pet_mean            0
p_seasonality       0
frac_snow           0
aridity             0
high_prec_freq      0
high_prec_dur       0
high_prec_timing    0
low_prec_freq       0
low_prec_dur        0
low_prec_timing     0
q_mean              0
runoff_ratio        0
slope_fdc           3
baseflow_index      0
stream_elas         0
Q5                  0
Q95                 0
high_q_freq         0
high_q_dur          0
low_q_freq          0
low_q_dur           0
zero_q_freq         0
hfd_mean            0
gauge_lat           0
gauge_lon           0
elev_mean           2
area                0
dom_land_cover      0
dtype: int64

In [31]:
# Remove NaN values
nan_cols = [col for col in filtered_attr.columns if filtered_attr.isna().sum()[col] > 0]
print(nan_cols)

for nan_col in nan_cols:
  filtered_attr[nan_col] = filtered_attr[nan_col].fillna(filtered_attr[nan_col].mean())

['slope_fdc', 'elev_mean']


<ipython-input-31-bd6338c072e3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_attr[nan_col] = filtered_attr[nan_col].fillna(filtered_attr[nan_col].mean())


In [32]:
BasicInputStaticProps = filtered_attr.to_numpy()
NpropperTimeStatic = len(filtered_attr.columns) - 1
print('NpropperTimeStatic', NpropperTimeStatic)
print(filtered_attr.columns)

NpropperTimeStatic 29
Index(['gauge_id', 'p_mean', 'pet_mean', 'p_seasonality', 'frac_snow',
       'aridity', 'high_prec_freq', 'high_prec_dur', 'high_prec_timing',
       'low_prec_freq', 'low_prec_dur', 'low_prec_timing', 'q_mean',
       'runoff_ratio', 'slope_fdc', 'baseflow_index', 'stream_elas', 'Q5',
       'Q95', 'high_q_freq', 'high_q_dur', 'low_q_freq', 'low_q_dur',
       'zero_q_freq', 'hfd_mean', 'gauge_lat', 'gauge_lon', 'elev_mean',
       'area', 'dom_land_cover'],
      dtype='object')


# Save Datasets

In [33]:
if CAMELS_COMBINED:
  input_p.to_csv(root_path + "/BasicInputTimeSeries_gb_combined.csv")
  filtered_attr.to_csv(root_path + "/BasicInputStaticProps_gb_combined.csv")
  np.save(root_path + "/BasicInputTimeSeries_gb_combined", BasicInputTimeSeries)
  np.save(root_path + "/BasicInputStaticProps_gb_combined", BasicInputStaticProps)

else:
  input_p.to_csv(root_path + "/BasicInputTimeSeries_gb.csv")
  filtered_attr.to_csv(root_path + "/BasicInputStaticProps_gb.csv")
  np.save(root_path + "/BasicInputTimeSeries_gb", BasicInputTimeSeries)
  np.save(root_path + "/BasicInputStaticProps_gb", BasicInputStaticProps)

In [34]:
from datetime import datetime
EndDate = datetime.strptime(EndDate, '%Y-%m-%d')
EndDate = np.datetime64(EndDate)
InitialDate = datetime.strptime(InitialDate, '%Y-%m-%d')
InitialDate = np.datetime64(InitialDate)

print(type(EndDate))
int(((EndDate-InitialDate + np.timedelta64(1, 'D'))/np.timedelta64(1, 'D')))

<class 'numpy.datetime64'>


7031

In [35]:
meta_data = {
    'Nloc': Nloc,
    'locs': locs.tolist(),
    'loc_names': attr_p['gauge_name'].tolist(),
    'BasicInputTimeSeries':{
      'fields': input_p.columns.values.tolist(),
      'index_fields': [BASIN_COL, DATE_COL],
      'initial_date': str(InitialDate),
      'end_date': str(EndDate),
      'time_delta': str(BasicInputTimeSeries[1, 1] - BasicInputTimeSeries[0, 1]),
      'time_steps': int(((EndDate-InitialDate + np.timedelta64(1, 'D'))/np.timedelta64(1, 'D'))),
    },
    'BasicInputStaticProps': {
        'fields': filtered_attr.columns.values.tolist(),
        'index_fileds': ['gauge_id'],
    },
    'NpropperTimeStatic': len(filtered_attr.columns) - 1
}

In [36]:
if CAMELS_COMBINED:
  with open(root_path + "/metadata_gb_combined.json", 'w') as outfile:
    json.dump(meta_data, outfile, indent='\t')

else:
  with open(root_path + "/metadata_gb.json", 'w') as outfile:
    json.dump(meta_data, outfile, indent='\t')

In [37]:
BasicInputTimeSeries

array([[Timestamp('1989-10-02 00:00:00'), 1001, 0.02, 8.5,
        1.9407891071608745],
       [Timestamp('1989-10-02 00:00:00'), 2001, 0.0, 7.6, 0.21],
       [Timestamp('1989-10-02 00:00:00'), 2002, 0.0, 7.22,
        1.9407891071608745],
       ...,
       [Timestamp('2008-12-31 00:00:00'), 102001, 0.0, -0.91, 1.14],
       [Timestamp('2008-12-31 00:00:00'), 106001, 0.0, 1.89, 2.33],
       [Timestamp('2008-12-31 00:00:00'), 107001, 1.07, -1.94, 0.94]],
      dtype=object)

In [38]:
type(BasicInputTimeSeries[0,3])

float

In [39]:
str(InitialDate)

'1989-10-02T00:00:00.000000'

In [40]:
str(BasicInputTimeSeries[1, 1] - BasicInputTimeSeries[0, 1])
print(BasicInputTimeSeries[1,1])
print(BasicInputTimeSeries[0,1])

2001
1001


In [41]:
meta_data

{'Nloc': 671,
 'locs': [1001,
  2001,
  2002,
  3003,
  4001,
  4003,
  4005,
  4006,
  5003,
  6007,
  6008,
  7001,
  7002,
  7003,
  7005,
  7006,
  8004,
  8005,
  8006,
  8009,
  8011,
  8013,
  9002,
  9003,
  9004,
  9005,
  10002,
  10003,
  11001,
  11003,
  11004,
  12001,
  12002,
  12005,
  12006,
  12007,
  12008,
  12009,
  13001,
  13004,
  13005,
  13007,
  13008,
  14001,
  14002,
  14005,
  15006,
  15007,
  15010,
  15011,
  15012,
  15013,
  15014,
  15016,
  15021,
  15023,
  15024,
  15025,
  15030,
  15039,
  16001,
  16003,
  16004,
  17001,
  17003,
  17004,
  17005,
  17015,
  17018,
  18001,
  18002,
  18003,
  18008,
  18010,
  18011,
  18014,
  18017,
  18018,
  19001,
  19006,
  19010,
  19017,
  19020,
  20002,
  20003,
  20007,
  21003,
  21006,
  21008,
  21009,
  21011,
  21012,
  21013,
  21015,
  21016,
  21017,
  21018,
  21022,
  21023,
  21024,
  21026,
  21027,
  22001,
  22006,
  22007,
  22009,
  23001,
  23004,
  23006,
  23007,
  23008,
  230